In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
from sklearn.decomposition import TruncatedSVD
import re
%matplotlib inline

pd.set_option('display.max_columns', 300)

In [11]:
# ==================
# Constant
# ==================
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"

In [12]:
# =====================
# Settings
# =====================
SAVE_PATH = "../output/fe/fe008.feather"

In [13]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
fe004 = pd.read_feather("../output/fe/fe004.feather")

In [14]:
fe004_train = fe004.iloc[:len(train)].copy()
fe004_test = fe004.iloc[len(train):].reset_index(drop=True)
fe004_train["Score"] = train["Score"]

In [15]:
# target_encoding
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
SEED = 0
N_SPLITS = 5
SHUFFLE = True
cat_cols = fe004_test.columns
target_cols = []
kf = KFold(n_splits=N_SPLITS,random_state = SEED,shuffle=SHUFFLE)
for c in cat_cols:
    print(c)
    fe004_train[f"{c}_target_encoding"] = np.nan
    test[f"{c}_target_encoding"] = np.nan
    for fold, (train_idx, valid_idx) in enumerate(kf.split(fe004_train,fe004_train["Score"])):
        x_train_dict = fe004_train.iloc[train_idx][["Score",c]].groupby(by=c)["Score"].mean().to_dict()
        fe004_train.loc[valid_idx,f"{c}_target_encoding"] = fe004_train.loc[valid_idx,c].map(x_train_dict)
    train_dict = fe004_train[["Score",c]].groupby(by=c)["Score"].mean().to_dict()
    fe004_test[f"{c}_target_encoding"] = fe004_test[c].map(train_dict)
    target_cols.append(f"{c}_target_encoding")

Gerres_0
Gerres_1
Gerres_2
Gerres_3
Gerres_4
Gerres_5
Gerres_6
Gerres_7
Gerres_8
Gerres_9
Gerres_10
Gerres_11
Gerres_12
Gerres_13
Gerres_14
Gerres_15
Gerres_16
Gerres_17
Gerres_18
Gerres_19
Gerres_20
Gerres_21
Gerres_22
Gerres_23
Gerres_24
Gerres_25
Gerres_26
Gerres_27
Gerres_28
Gerres_29
Gerres_30
Gerres_31
Gerres_32
Gerres_33
Gerres_34
Gerres_35
Gerres_36
Gerres_37
Gerres_38
Gerres_39
Gerres_40
Gerres_41
Gerres_42
Gerres_43
Gerres_44
Gerres_45
Gerres_46
Gerres_47


In [17]:
train = pd.concat([fe004_train[target_cols],fe004_test[target_cols]]).reset_index(drop=True)

In [19]:
train["genre_target_mean"] = train[target_cols].mean(axis=1)
train["genre_target_std"] = train[target_cols].std(axis=1)

In [20]:
train = reduce_mem_usage(train)
train.to_feather(SAVE_PATH)

Memory usage of dataframe is 4.74 MB
column =  50
0
Memory usage after optimization is: 2.37 MB
Decreased by 50.0%


In [21]:
train

,Gerres_0_target_encoding,Gerres_1_target_encoding,Gerres_2_target_encoding,Gerres_3_target_encoding,Gerres_4_target_encoding,Gerres_5_target_encoding,Gerres_6_target_encoding,Gerres_7_target_encoding,Gerres_8_target_encoding,Gerres_9_target_encoding,Gerres_10_target_encoding,Gerres_11_target_encoding,Gerres_12_target_encoding,Gerres_13_target_encoding,Gerres_14_target_encoding,Gerres_15_target_encoding,Gerres_16_target_encoding,Gerres_17_target_encoding,Gerres_18_target_encoding,Gerres_19_target_encoding,Gerres_20_target_encoding,Gerres_21_target_encoding,Gerres_22_target_encoding,Gerres_23_target_encoding,Gerres_24_target_encoding,Gerres_25_target_encoding,Gerres_26_target_encoding,Gerres_27_target_encoding,Gerres_28_target_encoding,Gerres_29_target_encoding,Gerres_30_target_encoding,Gerres_31_target_encoding,Gerres_32_target_encoding,Gerres_33_target_encoding,Gerres_34_target_encoding,Gerres_35_target_encoding,Gerres_36_target_encoding,Gerres_37_target_encoding,Gerres_38_target_encoding,Gerres_39_target_encoding,Gerres_40_target_encoding,Gerres_41_target_encoding,Gerres_42_target_encoding,Gerres_43_target_encoding,Gerres_44_target_encoding,Gerres_45_target_encoding,Gerres_46_target_encoding,Gerres_47_target_encoding,genre_target_mean,genre_target_std
0,6.715799,6.741415,6.488505,6.486294,6.492238,6.704346,6.525728,6.482156,6.945187,6.496430,6.456353,6.616793,6.489841,6.485314,6.533035,6.475589,6.494848,6.488101,6.510961,6.449357,6.475790,6.486294,6.484897,6.474660,6.536549,6.458885,6.449357,6.490579,6.481687,6.478254,6.482084,6.431323,6.485590,6.437709,6.616793,6.464848,6.473041,6.403351,6.449357,6.591000,6.475790,6.478254,6.450614,6.485643,6.494055,6.489816,6.492446,6.492781,6.512286,0.094118
1,6.396618,6.420223,6.467506,6.465743,6.470933,6.675149,6.507346,6.462569,6.924893,6.474135,6.432249,6.440622,6.472802,6.464006,6.513440,6.451845,6.475338,6.939231,6.486434,6.836474,6.452090,6.465743,6.462163,6.454873,6.509090,6.439215,6.836474,6.467724,6.461929,6.458815,6.461371,6.830073,6.466118,6.421853,6.440622,6.442722,6.453930,6.389953,6.836474,6.463954,6.456093,6.458815,6.430627,6.468154,6.473635,6.467739,6.471852,6.472563,6.514421,0.141207
2,6.715890,6.425461,6.477153,6.474637,6.481780,6.683548,6.515375,6.471273,6.395159,6.483449,6.441239,6.461399,6.479656,6.475168,6.526044,6.461864,6.485921,6.478171,6.498347,6.439409,6.460486,6.474637,6.475952,6.465253,6.519397,6.447505,6.439409,6.481311,6.473394,6.468873,6.472237,6.420724,6.476184,6.864810,6.461399,6.452745,6.462726,6.978848,6.439409,6.476130,6.812250,6.468873,6.437580,6.476784,6.483528,6.478678,6.481925,6.482650,6.504263,0.113709
3,6.687151,6.420223,6.467506,6.465743,6.470933,6.356147,6.507346,6.462569,6.389879,6.474135,6.432249,6.440622,6.472802,6.464006,6.513440,6.451845,6.475338,6.468005,6.486434,6.424632,6.452090,6.465743,6.462163,6.454873,6.509090,6.439215,6.424632,6.467724,6.461929,6.458815,6.461371,6.414437,6.466118,6.421853,6.440622,6.442722,6.453930,6.955209,6.424632,6.463954,6.812619,6.458815,6.430627,6.468154,6.473635,6.467739,6.471852,6.472563,6.477669,0.097187
4,6.687151,6.699973,6.467506,6.465743,6.470933,6.356147,6.507346,6.462569,6.389879,6.474135,6.432249,6.605862,6.472802,6.464006,6.513440,6.451845,6.335328,6.468005,6.486434,6.424632,6.452090,6.465743,6.462163,6.859750,6.509090,6.439215,6.424632,6.467724,6.461929,6.458815,6.461371,6.414437,6.466118,6.421853,6.605862,6.933550,6.453930,6.389953,6.424632,6.463954,6.456093,6.458815,6.838086,6.468154,6.473635,6.467739,6.471852,6.472563,6.495411,0.119662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12416,6.403594,6.426352,6.478765,6.475980,6.482296,6.364713,6.517291,6.473735,6.400305,6.485092,6.444230,6.603578,6.480713,6.475166,6.525651,6.464828,6.485855,6.478421,6.499189,6.438392,6.463227,6.475980,6.568830,6.465428,6.117609,6.450727,6.438392,6.480322,6.47